In [1]:
import random
import pickle
import re

wordnet = {}
with open("./data/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\admin\AppData\Local\Temp\ipykernel_5944\3047796988.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [3]:
EDA('와플대학은 대학 중에 최고다!')

['와플대학은 대학 중에 최고다!',
 '대학 와플대학은 중에 최고다!',
 '와플대학은 대 대학 중에 최고다!',
 '와플대학은 학 중에 최고다!',
 '와플대학은 학 중에 최고다!',
 '와플대학은 카 중에 최고다!',
 '와플대학은 대학 최고다! 중에',
 '와플대학은 대 대학 중에 최고다!',
 '와플대학은 대학 최고다!',
 '와플대학은 대학 중에 최고다!']

In [3]:
import pandas as pd

In [38]:
df = pd.read_excel('./data/긍정_일상문.xlsx')
df

,word,emotion,emo_num
0,유재석 오라버니 해피투게더 봤어요,흥미,1
1,우리모두 준혁이성과 LG선수들이 선전할수 있게 기를 넣어줍시다.,기쁨,1
2,웃긴거 알죠?~ㅋㅋㅇ ㅏ~,기쁨,1
3,덕분에 주방에서 요리하는게 넘 좋아졌어요,감사,1
4,마지막 순간 손가락으로 총을 만들어 쏘실때에는 제가 직접 맞는 것 같습니다.,감사,1
...,...,...,...
7040,"봐봐, 이거 너무 복실복실하지?",편안,1
7041,"하긴 남자애니까. 별로 관심 없으려나? 그래도 여기 남성 의류 말야, 평판이 꽤 좋...",만족,1
7042,이럴 땐 진짜 질색하는 표정이 된다니깐. 감탄스럽기까지 해.,불만,1
7043,"너무 대놓고 그러면 역효과잖니? 난 지금 이대로면 충분한걸. ……하긴, 옛날엔 페어...",만족,1


In [40]:
df = df.dropna()

In [41]:
df['emotion'].unique()

array(['흥미', '기쁨', '감사', '행복', '만족', '편안', '슬픔', '조롱', '불편', '궁금', '걱정',
       '보통', '답답', '불만', '분노', '당황', '허무함', '짜증', '무서움', '불안', '상처'],
      dtype=object)

In [42]:
# emotion 값에 따라 값 부여
emo_ = []
for i in df['emotion']:
    emo_.append(i)
    
    
emo_num = []

for i in range(0,len(emo_)):
    if emo_[i] == '분노' :
        emo_num.append(6)
    elif emo_[i] == '슬픔':
        emo_num.append(7)
    elif emo_[i] == '당황':
        emo_num.append(8)
    elif emo_[i] == '상처':
        emo_num.append(9)
    elif emo_[i] == '불안':
        emo_num.append(10)
    elif emo_[i] == '짜증':
        emo_num.append(11)
    elif emo_[i] == '답답':
        emo_num.append(12)
    elif emo_[i] == '허무함':
        emo_num.append(13)
    elif emo_[i] == '조롱':
        emo_num.append(14)
    elif emo_[i] == '불편':
        emo_num.append(15)
    elif emo_[i] == '걱정':
        emo_num.append(16)
    elif emo_[i] == '불만':
        emo_num.append(17)
    elif emo_[i] == '무서움':
        emo_num.append(18)
    elif emo_[i] == '편안':
        emo_num.append(0)
    elif emo_[i] == '기쁨':
        emo_num.append(1)
    elif emo_[i] == '행복':
        emo_num.append(2)
    elif emo_[i] == '감사':
        emo_num.append(3)
    elif emo_[i] == '만족':
        emo_num.append(4)
    elif emo_[i] == '흥미':
        emo_num.append(5)
    elif emo_[i] == '보통':
        emo_num.append(19)
    else :
        emo_num.append(20)

In [43]:
df = df.reset_index(drop=True)

In [44]:
df['emo_num'] = pd.DataFrame(emo_num)
df

,word,emotion,emo_num
0,유재석 오라버니 해피투게더 봤어요,흥미,5
1,우리모두 준혁이성과 LG선수들이 선전할수 있게 기를 넣어줍시다.,기쁨,1
2,웃긴거 알죠?~ㅋㅋㅇ ㅏ~,기쁨,1
3,덕분에 주방에서 요리하는게 넘 좋아졌어요,감사,3
4,마지막 순간 손가락으로 총을 만들어 쏘실때에는 제가 직접 맞는 것 같습니다.,감사,3
...,...,...,...
7034,"봐봐, 이거 너무 복실복실하지?",편안,0
7035,"하긴 남자애니까. 별로 관심 없으려나? 그래도 여기 남성 의류 말야, 평판이 꽤 좋...",만족,4
7036,이럴 땐 진짜 질색하는 표정이 된다니깐. 감탄스럽기까지 해.,불만,17
7037,"너무 대놓고 그러면 역효과잖니? 난 지금 이대로면 충분한걸. ……하긴, 옛날엔 페어...",만족,4


In [12]:
df.to_excel('./data/긍정일상문.xlsx', index=False)

In [21]:
df_h = df[df['emotion']=='허무함']
# df_h = df_h.reset_index(drop=True)
df_h

,word,emotion,emo_num
2819,이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다...,허무함,13
3542,알려주시면 감사하겠습니다ㅠㅠ!,허무함,13
4596,지금까지?여기 피시방알바하면서 젤많이팔았다 20만원.ㅋ,허무함,13
5011,허허허 웃음밖에안나오심,허무함,13
5261,솔직히 나만큼 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ,허무함,13
5918,30분 졸았네...ㅋㅋ,허무함,13
6569,좋죠. 세월이 지날수록 세상이 편해지기는 하는 것 같은데 그렇다고 세상이 더 좋아졌...,허무함,13
6714,알았어 알았어.. 큭큭.. 우와~ 니가 우리 오빨..,허무함,13
6884,"아하하, 하긴.",허무함,13


In [18]:
len(df_h)

9

In [22]:
#  df_h[0][0]

In [24]:
eda =[]
for i in df_h['word']:
    # print(i)
    a = EDA(i)
    b = list(set(a))
    eda.append(b)

In [25]:
eda

[['이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
  '이 문장을 읽고 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
  '이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 돌아봤답니다....^^a 잠시',
  '이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 잠시 돌아봤답니다....^^a',
  '이 이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
  '인생을 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 이 잠시 돌아봤답니다....^^a',
  '이 짧은 문장을 읽고 하루종일...전..27년이라는제 그날 인생을 잠시 돌아봤답니다....^^a'],
 [' 알려주시면 감사하겠습니다ㅠㅠ!', '감사하겠습니다ㅠㅠ! 알려주시면', '알려주시면', '알려주시면 감사하겠습니다ㅠㅠ!'],
 ['피시방알바하면서 지금까지?여기 젤많이팔았다 20만원.ㅋ',
  '지금까지?여기 20만원.ㅋ 젤많이팔았다 피시방알바하면서',
  '20만원.ㅋ 피시방알바하면서 젤많이팔았다 지금까지?여기',
  '지금까지?여기 피시방알바하면서 젤많이팔았다 20만원.ㅋ',
  '지금까지?여기 젤많이팔았다 20만원.ㅋ'],
 ['허허허 웃음밖에안나오심'],
 ['솔직히 나만큼 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 나만큼 너좋아해준 람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 나만큼 사 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 나만큼 너좋아해준 사 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 나만큼 너좋아해준 뻥 없다고 사람 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 뻥 너좋아해준 사람 없다고 나만큼 찼네요 ㅋㅋㅋㅋㅋ',
  ' 솔직히 나만큼 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
  '솔직히 나만큼 너좋아해준 사 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ'],
 ['30분 졸았네...ㅋㅋ', '졸았네

In [26]:
augm = sum(eda, [])
augm

['이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
 '이 문장을 읽고 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
 '이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 돌아봤답니다....^^a 잠시',
 '이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 잠시 돌아봤답니다....^^a',
 '이 이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a',
 '인생을 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 이 잠시 돌아봤답니다....^^a',
 '이 짧은 문장을 읽고 하루종일...전..27년이라는제 그날 인생을 잠시 돌아봤답니다....^^a',
 ' 알려주시면 감사하겠습니다ㅠㅠ!',
 '감사하겠습니다ㅠㅠ! 알려주시면',
 '알려주시면',
 '알려주시면 감사하겠습니다ㅠㅠ!',
 '피시방알바하면서 지금까지?여기 젤많이팔았다 20만원.ㅋ',
 '지금까지?여기 20만원.ㅋ 젤많이팔았다 피시방알바하면서',
 '20만원.ㅋ 피시방알바하면서 젤많이팔았다 지금까지?여기',
 '지금까지?여기 피시방알바하면서 젤많이팔았다 20만원.ㅋ',
 '지금까지?여기 젤많이팔았다 20만원.ㅋ',
 '허허허 웃음밖에안나오심',
 '솔직히 나만큼 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 나만큼 너좋아해준 람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 나만큼 사 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 나만큼 너좋아해준 사 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 나만큼 너좋아해준 뻥 없다고 사람 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 뻥 너좋아해준 사람 없다고 나만큼 찼네요 ㅋㅋㅋㅋㅋ',
 ' 솔직히 나만큼 너좋아해준 사람 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '솔직히 나만큼 너좋아해준 사 없다고 뻥 찼네요 ㅋㅋㅋㅋㅋ',
 '30분 졸았네...ㅋㅋ',
 '졸았네...ㅋㅋ 30분',
 '세월이 세상이 편해

In [27]:
DF = pd.DataFrame(augm)
DF

,0
0,이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다...
1,이 문장을 읽고 하루종일...전..27년이라는제 인생을 잠시 돌아봤답니다....^^a
2,이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 돌아봤답니다......
3,이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 잠시 돌아봤답니다....^^a
4,이 이 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 인생을 잠시 돌아봤답...
5,인생을 짧은 문장을 읽고 그날 하루종일...전..27년이라는제 이 잠시 돌아봤답니다...
6,이 짧은 문장을 읽고 하루종일...전..27년이라는제 그날 인생을 잠시 돌아봤답니다...
7,알려주시면 감사하겠습니다ㅠㅠ!
8,감사하겠습니다ㅠㅠ! 알려주시면
9,알려주시면


In [30]:
DF.to_excel('./data/긍정_허무함.xlsx', index=False)

#### augmentation 함수로 정리

In [28]:
def AUG(emo):
    df_d = df[df['emotion']==emo]
    df_d = df_d.reset_index(drop=True)

    eda =[]
    for i in df_d['word']:
        # print(i)
        a = EDA(i)
        b = list(set(a))
        eda.append(b)
        
    augm = sum(eda, [])
    return augm


In [47]:
df_ = AUG('행복')
df_

['활동혜택으로 오션월드 입장권 2매까지 폴인러버!',
 '폴인러버! 오션월드 입장권 2매까지 주는 활동혜택으로',
 '활동혜택으로 오션월드 입장권 2매까지 주는 폴인러버!',
 '활동혜택으로 입장권 오션월드 2매까지 주는 폴인러버!',
 '활동혜택으로 주는 입장권 2매까지 오션월드 폴인러버!',
 '오션월드 입장권 2매까지 주는 폴인러버!',
 '원호 형님 형님이 올해 온다고 해서 얼마나 기뻤는지 몰라요',
 '형님 올해 우리팀으로 온다고 해서 얼마나 기뻤는지 몰라요',
 '원호 형님 형님이 기뻤는지 우리팀으로 온다고 해서 얼마나 올해 몰라요',
 '원호 형님 해서 올해 우리팀으로 온다고 형님이 얼마나 기뻤는지 몰라요',
 '원호 형님 형님이 올해 우리팀으로 온다고 해서 기뻤는지 얼마나 몰라요',
 '원호 형님 형님이 올해 우리팀으로 온다고 해서 얼마나 기뻤는지 몰라요',
 '앞으로 언제나 캐럿들이 함께 있을게요!',
 '앞으로 언제나 함께 있을게요!',
 ' 앞으로 언제나 캐럿들이 함께 있을게요!',
 '캐럿들이 언제나 앞으로 함께 있을게요!',
 '함께 언제나 캐럿들이 앞으로 있을게요!',
 '있을게요! 언제나 캐럿들이 함께 앞으로',
 ' 정말 행복해요.',
 '행복해요. 정말',
 '정말 행복해요.',
 '정말',
 '오빠 오늘 하루도 행쇼~♥',
 '그럼 행쇼~♥ 오늘 하루도 오빠',
 '오 그럼 오빠 오늘 하루도 행쇼~♥',
 '그럼 오빠 오늘 하루도 오 행쇼~♥',
 '그럼 오빠 오늘 하루도 행쇼~♥',
 '그럼 오빠 오 하루도 행쇼~♥',
 '하루도 오빠 오늘 그럼 행쇼~♥',
 ' 그럼 오빠 오늘 하루도 행쇼~♥',
 '그럼 오빠 늘 하루도 행쇼~♥',
 '그럼 오빠 하루도 오늘 행쇼~♥',
 '오늘은 한도전하는날~ 무',
 '오늘은 무 무 한도전하는날~',
 '  오늘은 무 한도전하는날~',
 '오늘은 무 한도전하는날~',
 '무 오늘은 무 한도전하는날~',
 '연말에 배우 볼수 있다 ...조으당~~~',
 '연말에 박보검 배 볼수 있다 .

In [48]:
df_ = pd.DataFrame(df_)
df_.to_excel('./data/행복증강.xlsx', index=False)